<a href="https://colab.research.google.com/github/ColdCoffee21/Foundations-of-Data-Science/blob/master/Kaggle_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all, you run the following code and upload your kaggle.json.

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"code1110","key":"efed5070a812d29675720408784429b0"}'}

Next, run the following code to set the path.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Then, run the following to install kaggle!

In [ ]:
!pip install kaggle

Run the following to add access permission to yourself to download data.

You can also see the list of datasets you can donwload!

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  
------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
himanshupoddar/zomato-bangalore-restaurants             Zomato Bangalore Restaurants                       88MB  2019-03-31 09:48:43           3795  
thegurus/spanish-high-speed-rail-system-ticket-pricing  Spanish High Speed Rail tickets pricing - Renfe    27MB  2019-05-10 06:50:30            905  
crisparada/brazilian-cities                             Brazilian Cities                                  935KB  2019-05-24 20:49:21           1591  
chicago/chicago-copa-cases                              Chicago COPA Cases                                  3MB  2019-04-05 00:00:32           1729  
taniaj/australian-election-2019-tweets                  Australian Election 2019 Tweets             

As you see, you can download a lot of data here. This time we just download the data in MNIST competition. Run the following to do it.

In [ ]:
!kaggle competitions download -c digit-recognizer

 89% 65.0M/73.2M [00:00<00:00, 87.9MB/s]
100% 73.2M/73.2M [00:00<00:00, 115MB/s] 
100% 48.8M/48.8M [00:00<00:00, 51.5MB/s]

  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 76.2MB/s]


You can find the code to install data in many competitions in data description section (shown as API ...).  

The following is basically the same as what you would do in kaggle kernel, except that we need to install keras as it is not available in default in google colab.


In [ ]:
!pip install -q keras
import keras

Using TensorFlow backend.


Note that you are in /content.

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import further libraries
import matplotlib.pyplot as plt
import seaborn as sns

# keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split

As you are in /content, loading data can be done in the following.

In [ ]:
# load training & test datasets
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

The rest is just modeling:D

In [ ]:
# pandas to numpy
y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

del train

# normalize
X_train = X_train/255.0
test = test/255.0

# reshape the data so that the data
# represents (label, img_rows, img_cols, grayscale)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# one-hot vector as a label (binarize the label)
y_train = to_categorical(y_train, num_classes=10)

In [ ]:
# Three steps to create a CNN
# 1. Convolution
# 2. Activation
# 3. Pooling
# Repeat Steps 1,2,3 for adding more hidden layers

# 4. After that make a fully connected network
# This fully connected network gives ability to the CNN
# to classify the samples

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

W0621 12:42:48.743506 140195555882880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0621 12:42:48.787888 140195555882880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0621 12:42:48.796316 140195555882880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0621 12:42:48.840509 140195555882880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0621 12:42:48.843660 1401955558

In [ ]:
# compile model
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

W0621 12:42:59.599435 140195555882880 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
# cross validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=1220)

In [ ]:
# data argumentation
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
train_generator = gen.flow(X_train, y_train, batch_size=64)

In [ ]:
# learning rate
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
# model training
model.fit_generator(train_generator, epochs=30, validation_data = (X_val, y_val), verbose=2, steps_per_epoch=X_train.shape[0]/36,
                                       callbacks=[learning_rate_reduction])

W0621 12:43:08.377925 140195555882880 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
 - 25s - loss: 0.1137 - acc: 0.9645 - val_loss: 0.0674 - val_acc: 0.9814
Epoch 2/30
 - 20s - loss: 0.0461 - acc: 0.9854 - val_loss: 0.0664 - val_acc: 0.9817
Epoch 3/30
 - 20s - loss: 0.0405 - acc: 0.9881 - val_loss: 0.0557 - val_acc: 0.9886
Epoch 4/30
 - 19s - loss: 0.0350 - acc: 0.9896 - val_loss: 0.0430 - val_acc: 0.9902
Epoch 5/30
 - 20s - loss: 0.0303 - acc: 0.9907 - val_loss: 0.0992 - val_acc: 0.9793
Epoch 6/30
 - 20s - loss: 0.0257 - acc: 0.9926 - val_loss: 0.0626 - val_acc: 0.9876
Epoch 7/30
 - 19s - loss: 0.0260 - acc: 0.9926 - val_loss: 0.0289 - val_acc: 0.9938
Epoch 8/30
 - 19s - loss: 0.0223 - acc: 0.9933 - val_loss: 0.0836 - val_acc: 0.9879
Epoch 9/30
 - 20s - loss: 0.0259 - acc: 0.9929 - val_loss: 0.0563 - val_acc: 0.9888
Epoch 10/30
 - 19s - loss: 0.0207 - acc: 0.9941 - val_loss: 0.0354 - val_acc: 0.9921

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 11/30
 - 19s - loss: 0.0155 - acc: 0.9956 - val_loss: 0.0387 - val_acc: 

In [ ]:
# model prediction on test data
predictions = model.predict_classes(test, verbose=0)

In [ ]:
# make a submission file
submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)),
    "Label": predictions})
submissions.to_csv("my_submission.csv", index=False, header=True)

Finally by running the following command, you can submit your file to kaggle from google colab!

In [ ]:
# submit the file to kaggle
!kaggle competitions submit digit-recognizer -f my_submission.csv -m "Yeah! I submit my file through the Google Colab!"

Now you can go back to Kaggle to see where you are on the Leaderboard:D Enjoy kaggle more with google colab!